<h1 style="color:green" align='center'> Data Science Regression Project: Predicting Home Prices in Bangalore </h1>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

import warnings
warnings.filterwarnings('ignore')

<h2 style='color:blue'>1. Data Load: Load Bangalore home price into a dataframe:</h2>

In [2]:
df = pd.read_csv("./../datasets/bengaluru_house_prices.csv")
print(df.shape)
df.head(3)

(13320, 9)


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00


#### **Check for null values in the dataframe for each column.**

In [3]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [4]:
df.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

#### **Drop features/columns that are not required to build our model and assign it into the new dataframe df1.**

In [5]:
df1 = df.drop(['area_type', 'availability', 'society', 'balcony'], axis='columns')
df1.head(3)

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00


<h2 style='color:blue'>2. Data Clearning and Handling: </h2>

#### **Check the null value after dropping/removing some features.**

In [6]:
df1.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [7]:
df1.shape

(13320, 5)

#### **Drop all the null values as we have big dataset it will not affect the outcome much.**

In [8]:
df2 = df1.dropna()
df2.shape

(13246, 5)

#### **Check for null values after new dataframe df2 just to make sure there are none.**

In [9]:
df2.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

<h2 style='color: blue'>3. Feature Engineering(1):</h2>

In [10]:
df2.head(3)

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00


**Check the unique value of feature 'size'.**

In [11]:
df2['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

#### **Add new features called 'bhk'(bed hall kitchen). Split the feature 'size' into two and take the first value which is the number we want.**

**"2 BHK".split(" ") will split the categorical value '2 BHK'into [2, 'BHK'] but we want the first value so "2 BHK".split(" ")[0] will return only 2.**


In [12]:
df2['bhk'] = df2['size'].apply(lambda x: int(x.split(' ')[0]))
df2.head(3)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3


**Drop size feature from the dataframe.**

In [13]:
df3 = df2.drop('size', axis=1)
df3.head(3)

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3


In [14]:
df3.shape

(13246, 5)

**Now check the values of bhk to confirm it's only numerical as we expect.**

In [15]:
df3['bhk'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

**Explore total_sqft. feature by checking if all the values are float type or not by using a user defined function.**

In [16]:
def is_float(data):
    try:
        float(data)
        return True
    except: 
        return False

**Check the total rows where the total_sqft features values are float.**

In [17]:
df3[df['total_sqft'].apply(is_float)].shape

(13056, 5)

**Explore the rows where total_sqft features values are not float and how to dealt with it.**

In [18]:
df3[~df['total_sqft'].apply(is_float)].head(10)

,location,total_sqft,bath,price,bhk
30,Yelahanka,2100 - 2850,4.0,186.000,4
122,Hebbal,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,1042 - 1105,2.0,54.005,2
165,Sarjapur,1145 - 1340,2.0,43.490,2
188,KR Puram,1015 - 1540,2.0,56.800,2
410,Kengeri,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,1195 - 1440,2.0,63.770,2
648,Arekere,4125Perch,9.0,265.000,9
661,Yelahanka,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,3090 - 5002,4.0,445.000,4


**Transform the range value such as '2100-2980' to a single value by taking means and drop the rest by working around through user define function.
'2000-2100'.split('-') will return ['2000', '2100']**

In [19]:
def range_to_mean(data):
    ret_val = data.split('-')
    if len(ret_val) == 2:
        return (float(ret_val[0])+float(ret_val[1]))/2
    try: 
        return float(data)
    except:
        return None
    

In [20]:
df4 = df3.copy()

In [21]:
df4['total_sqft'] = df4['total_sqft'].apply(range_to_mean)
df4['total_sqft'].values

array([1056., 2600., 1440., ..., 1141., 4689.,  550.])

In [22]:
df4.shape

(13246, 5)

**There are some elements from df4['total_sqft'] features that are NaN, we will remove those.**

In [23]:
df4 = df4[df4['total_sqft'].notnull()]
df4.shape

(13200, 5)

In [24]:
df4.loc[30, ['total_sqft']]

total_sqft    2475.0
Name: 30, dtype: object

In [25]:
df4.loc[30, 'total_sqft']

2475.0

<h2 style='color: blue'> 3. Feature Engineering(2):</h2>

**Add new feature called 'price_per_sqft' to check analyze the price of an apartment per square feet.**

In [26]:
df5 = df4.copy()

df5['price_per_sqft'] = (df5['price']*100000)/df5['total_sqft']
df5.head(10)

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000
5,Whitefield,1170.0,2.0,38.00,2,3247.863248
6,Old Airport Road,2732.0,4.0,204.00,4,7467.057101
7,Rajaji Nagar,3300.0,4.0,600.00,4,18181.818182
8,Marathahalli,1310.0,3.0,63.25,3,4828.244275
9,Gandhi Bazar,1020.0,6.0,370.00,6,36274.509804


**Analyze price_per_sqft through statistical result.**

In [27]:
df5_stats = df5['price_per_sqft'].describe()
df5_stats

count    1.320000e+04
mean     7.920759e+03
std      1.067272e+05
min      2.678298e+02
25%      4.267701e+03
50%      5.438331e+03
75%      7.317073e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

**Strip or trim all the white spaces or blanks that may be present at the end and beginning of the value of location feature**

In [56]:
df6 = df5.copy()
df6['location'] = df5['location'].apply(lambda x: x.strip())

**Examine the location feature, which is a categorical value or variable. We will need to apply dimensionality reduction technique to reduce the number of location name if it is very large**

In [57]:
location_stats = df6['location'].value_counts(ascending=False)
location_stats

location
Whitefield                   533
Sarjapur  Road               392
Electronic City              304
Kanakpura Road               264
Thanisandra                  235
                            ... 
Rajanna Layout                 1
Subramanyanagar                1
Lakshmipura Vidyaanyapura      1
Malur Hosur Road               1
Abshot Layout                  1
Name: count, Length: 1287, dtype: int64

**Exploring the total numbers of unique location name we have**

In [58]:
location_stats.values.sum()

13200

In [59]:
print(type(location_stats))
len(location_stats)

<class 'pandas.core.series.Series'>


1287

In [60]:
print(len(location_stats[location_stats <=10]))
print(len(location_stats[location_stats >10]))

1047
240


In [61]:
location_stats_less_than_11 = location_stats[location_stats<=10]
print(len(location_stats_less_than_11))
location_stats_less_than_11

1047


location
BTM 1st Stage                10
Gunjur Palya                 10
Nagappa Reddy Layout         10
Sector 1 HSR Layout          10
Thyagaraja Nagar             10
                             ..
Rajanna Layout                1
Subramanyanagar               1
Lakshmipura Vidyaanyapura     1
Malur Hosur Road              1
Abshot Layout                 1
Name: count, Length: 1047, dtype: int64

<h2 style='color: blue'>3. Feature Engineering (Dimensionality Reduction): </h2>

**Any location having less than 10 data points should be tagged as 'other' location. This way number of categories can be reduced by huge amount. Later on when we apply 'one hot encoding', we will have fewer dummy variables.**

In [62]:
df7 = df6.copy()
df7.shape

(13200, 6)

In [63]:
df6.head()

,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000


In [64]:
df7['location'] = df6['location'].apply(lambda x: 'others' if x in location_stats_less_than_11 else x)

In [65]:
len(df7.location.unique())

241

<h2 style='color:blue'>4. Outlier Removal (Business Logic):</h2>

**As a data scientist when you have a conversation with your business manager (who has expertise in real estate), he will tell you that normally square ft per bedroom is 300 (i.e. 2 bhk apartment is minimum 600 sqft. If you have for example 400 sqft apartment with 2 bhk than that seems suspicious and can be removed as an outlier. We will remove such outliers by keeping our minimum thresold per bhk to be 300 sqft**

In [67]:
df7[(df7['total_sqft']/df7['bhk'])<300].head()

,location,total_sqft,bath,price,bhk,price_per_sqft
9,others,1020.0,6.0,370.0,6,36274.509804
45,HSR Layout,600.0,9.0,200.0,8,33333.333333
58,Murugeshpalya,1407.0,4.0,150.0,6,10660.980810
68,Devarachikkanahalli,1350.0,7.0,85.0,8,6296.296296
70,others,500.0,3.0,100.0,3,20000.000000


**From above table we saw that 'bhk' with 8 have 'total_sqft' value 1350 but 'bhk' with 6 have 'total_sqft' 1407 which is larger than 8 'bhk'. This shouldn't be the case so they are taken as error and have to be remove.**

In [72]:
df8 = df7[(df7['total_sqft']/df7['bhk']) >= 300]
print(df8.shape)
df8.head()

(12456, 6)


,location,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,1200.0,2.0,51.00,2,4250.000000


<h2 style='color:blue'>4. Outlier Removal (Standard Deviation and Mean):</h2>

In [70]:
df8.price_per_sqft.describe()

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64

**From above statistical result we learn that there is a wide variation between minimum and maximum value which can be due to outliers present in the dataset. We need to remove those outlier using standard deviationa and mean.**

In [77]:
len(df8[(df8.price_per_sqft > (df8.price_per_sqft.mean()+df8.price_per_sqft.std()))])

1122

In [76]:
len(df8[(df8.price_per_sqft < (df8.price_per_sqft.mean() - df8.price_per_sqft.std()))])

40

In [88]:
def removed_pp_sqft_outlier(df):
    new_df = pd.DataFrame()
    for _, loc_df in df.groupby('location'):
        avg = np.average(loc_df.price_per_sqft)
        stdev = np.std(loc_df.price_per_sqft)
        reduced_df = loc_df[(loc_df.price_per_sqft > (avg - 3*stdev)) & (loc_df.price_per_sqft <= (avg + 3*stdev))]
        new_df = pd.concat([new_df, reduced_df], ignore_index=True)
    return new_df
        

In [91]:
df9 = removed_pp_sqft_outlier(df8)
print(df9.shape)
df9.head()

(12266, 6)


,location,total_sqft,bath,price,bhk,price_per_sqft
0,1st Block Jayanagar,2850.0,4.0,428.0,4,15017.543860
1,1st Block Jayanagar,1630.0,3.0,194.0,3,11901.840491
2,1st Block Jayanagar,1875.0,2.0,235.0,3,12533.333333
3,1st Block Jayanagar,2400.0,4.0,450.0,4,18750.000000
4,1st Block Jayanagar,1000.0,3.0,60.0,2,6000.000000


In [80]:
np.average([1,1,1,1])

1.0

In [81]:
np.mean([1,1,1,1])

1.0